In [110]:
import re
import xml.etree.ElementTree as etree
from collections import defaultdict
from functools import reduce

In [3]:
import numpy as np
import pandas as pd

## Read wikipedia dump

In [6]:
root = etree.parse('../data/kowiki.xml').getroot()

In [109]:
prefix = '{http://www.mediawiki.org/xml/export-0.10/}'
keywords = ['title', 'id', 'revision/text']

In [132]:
context = defaultdict(dict)

In [133]:
for page in root[1:]:
    data = {keyword: reduce(lambda node, key: node.find(f'{prefix}{key}'), [page, *keyword.split('/')]).text for keyword in keywords}
    context[data['title']] = data

## Parser

In [186]:
regex_document = re.compile('\[\[(.[^:]+?)\]\]')
regex_attach = re.compile('\[\[파일:(.+?)\]\]')
regex_comment = re.compile('\[\*(.+?)\]')
regex_table = re.compile('\|(.+?)=(.+?)\n')
regex_link = re.compile('\[\[(.[^:\[\]]+?)\|(.[^:\[\]]+?)\]\]')
regex_del = re.compile('~~(.+?)~~')
regex_tag = re.compile('\<(.+?)\>')

## Get drama titles from channles

In [135]:
interested = ["대한민국의 텔레비전 드라마 목록/ㄱ", "대한민국의 텔레비전 드라마 목록/ㄴ", "대한민국의 텔레비전 드라마 목록/ㄷ", "대한민국의 텔레비전 드라마 목록/ㄹ", "대한민국의 텔레비전 드라마 목록/ㅁ", "대한민국의 텔레비전 드라마 목록/ㅂ", "대한민국의 텔레비전 드라마 목록/ㅅ", "대한민국의 텔레비전 드라마 목록/ㅇ", "대한민국의 텔레비전 드라마 목록/ㅈ", "대한민국의 텔레비전 드라마 목록/ㅊ", "대한민국의 텔레비전 드라마 목록/ㅋ", "대한민국의 텔레비전 드라마 목록/ㅌ", "대한민국의 텔레비전 드라마 목록/ㅍ", "대한민국의 텔레비전 드라마 목록/ㅎ", "대한민국의 텔레비전 드라마 목록/알파벳", "대한민국의 텔레비전 드라마 목록/숫자", ]

In [165]:
titles = set()

In [166]:
for inter in interested:
    document = context[inter]['revision/text']
    matches = regex_documnet.findall(document)
    
    for match in matches:
        name, *_ = match.split('|')
        name, *_ = name.split('#')
        if name != '대한민국의 텔레비전 드라마 목록':
            titles.add(name)

## Get metadata from document

In [201]:
columns = {
    '방송 기간': ['방송기간', '방송 기간'],
    '방송 횟수': ['횟수', '방송 횟수'],
    '장르': ['장르'],
    '채널': ['채널', '방송사', '방송 채널', '방송채널'],
    '제작사': ['제작사', '제작자', '제작'],
    '극본': ['극본', '대본', '각본'],
    '출연자': ['출연자', '출연', '출연진'],
}

In [202]:
data = defaultdict(dict)
notexists = []
notparser = []

In [203]:
for title in titles:
    try:
        document = context[title]['revision/text']                  
    except KeyError:
        notexists.append(title)
        
    matches = regex_table.findall(document)
    
    d = {key: '' for key in columns}
    
    for key_value in matches:
        key, value = map(str.strip, key_value)
        key = next((ckey for ckey, cvalues in columns.items() if any(cvalue in key for cvalue in cvalues)), False)
        if key:
            d[key] = value
    
    if not d:
        notparser.append(title)
    else:
        data[title] = d

## Create table from data

In [204]:
table_columns = list(columns.keys())

In [205]:
table = np.empty((0, len(table_columns) + 1))

In [ ]:
for title, values in data.items():
    d = [regex_tag.sub('', regex_comment.sub('', regex_del.sub(r'\1', regex_documnet.sub(r'\1', regex_link.sub(r'\1', regex_attach.sub('attach', values[column])))))).strip() 
         for column in table_columns]
    table = np.vstack((table, np.array([title, *d])))

In [ ]:
df = pd.DataFrame(table)
df.columns = ['title', *table_columns]

In [ ]:
df.head()

In [ ]:
df.to_csv('../results/namuwiki.csv', index=None)